In [126]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [128]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

%matplotlib inline

## Loading the data
We can now load the dataset into pandas using the read_csv() function. This converts the CSV file into a Pandas dataframe.

In [2]:
#Read in the csv file and convert to a Pandas dataframe
df = pd.read_csv("final_data.csv")

### Viewing the dataframe
We can get a quick sense of the size of our dataset by using the shape method. This returns a tuple with the number of rows and columns in the dataset.

In [3]:
df.head()

,player,team,name,position,height,age,appearance,goals,assists,yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,/david-de-gea/profil/spieler/59377,Manchester United,David de Gea,Goalkeeper,189.0,32.0,104,0.000000,0.000000,0.009585,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,/jack-butland/profil/spieler/128899,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.000000,0.000000,0.069018,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,/tom-heaton/profil/spieler/34130,Manchester United,Tom Heaton,Goalkeeper,188.0,37.0,4,0.000000,0.000000,0.000000,...,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,/lisandro-martinez/profil/spieler/480762,Manchester United,Lisandro Martínez,Defender Centre-Back,175.0,25.0,82,0.028090,0.056180,0.224719,...,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,/raphael-varane/profil/spieler/164770,Manchester United,Raphaël Varane,Defender Centre-Back,191.0,30.0,63,0.017889,0.017889,0.053667,...,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0


In [4]:
df.shape

(10754, 22)

## 1. Data Profiling:
Data profiling is a comprehensive process of examining the data available in an existing dataset and collecting statistics and information about that data. 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10754 entries, 0 to 10753
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player               10754 non-null  object 
 1   team                 10754 non-null  object 
 2   name                 10754 non-null  object 
 3   position             10754 non-null  object 
 4   height               10754 non-null  float64
 5   age                  10754 non-null  float64
 6   appearance           10754 non-null  int64  
 7   goals                10754 non-null  float64
 8   assists              10754 non-null  float64
 9   yellow cards         10754 non-null  float64
 10  second yellow cards  10754 non-null  float64
 11  red cards            10754 non-null  float64
 12  goals conceded       10754 non-null  float64
 13  clean sheets         10754 non-null  float64
 14  minutes played       10754 non-null  int64  
 15  days_injured         10754 non-null 

In [11]:
df.describe(include='float')

,height,age,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets
count,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000
mean,181.240353,26.041903,0.125554,0.086978,0.189757,0.004666,0.006826,0.131655,0.044881
std,6.969818,4.777629,0.235585,0.143351,0.432388,0.025232,0.081143,0.442335,0.924437
min,156.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,176.000000,22.000000,0.000000,0.000000,0.053191,0.000000,0.000000,0.000000,0.000000
50%,181.240353,26.000000,0.045969,0.040773,0.150250,0.000000,0.000000,0.000000,0.000000
75%,186.000000,29.000000,0.172263,0.133136,0.248276,0.000000,0.000000,0.000000,0.000000
max,206.000000,43.000000,11.250000,4.000000,30.000000,1.000000,6.923077,9.000000,90.000000


In [29]:
df.describe(include='int')

,appearance,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
count,10754.000000,10754.000000,10754.000000,10754.000000,10754.000000,1.075400e+04,1.075400e+04,10754.000000,10754.000000
mean,36.407011,2470.789381,117.961689,15.826297,1.960759,3.622971e+06,6.152606e+06,2.713223,0.307513
std,26.526541,2021.703271,175.206827,23.383606,3.743936,9.095410e+06,1.338988e+07,0.986356,0.461485
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,0.000000
25%,12.000000,660.000000,0.000000,0.000000,0.000000,3.000000e+05,4.500000e+05,2.000000,0.000000
50%,35.000000,2101.500000,37.000000,5.000000,1.000000,8.000000e+05,1.500000e+06,3.000000,0.000000
75%,59.000000,3968.000000,181.000000,24.000000,2.000000,3.000000e+06,5.000000e+06,4.000000,1.000000
max,107.000000,9510.000000,2349.000000,339.000000,92.000000,1.800000e+08,2.000000e+08,4.000000,1.000000


In [13]:
df.describe(include='object')

,player,team,name,position
count,10754,10754,10754,10754
unique,10754,374,10584,16
top,/david-de-gea/profil/spieler/59377,Daejeon Hana Citizen,Paulinho,Defender Centre-Back
freq,1,46,6,1821


#### 4. Relevance: 
Assess whether the data is appropriate and applicable for the intended analysis. Data that is not relevant can skew results and lead to incorrect conclusions.

**Key considerations for relevance include:**

> 1. Sample Appropriateness: Confirm that your data sample aligns with your analysis objectives. For instance, utilizing data from the Northern region will not yield accurate insights for the Western region of the Kingdom.
>
> 2. Variable Selection: Any column will not be relevant for our analysis, we can get rid of these using the drop() method. We will set the “axis” argument to 1 since we’re dealing with columns, and set the “inplace” argument to True to make the change permanent.


In [22]:
df.groupby(['name', 'team'])[['name', 'team']].count()

,,name,team
name,team,,
AJ Marcucci,New York Red Bulls,1,1
Aaron Appindangoyé,Sivasspor,1,1
Aaron Boupendza,Al-Shabab Club (Riyadh),1,1
Aaron Calver,Gwangju FC,1,1
Aaron Connolly,Hull City,1,1
...,...,...,...
Ömer Beyaz,VfB Stuttgart,1,1
Ömer Toprak,Antalyaspor,1,1
Örjan Nyland,RB Leipzig,1,1


In [4]:
df.drop(["player"],axis=1,inplace=True)

In [7]:
df.head(2)

,team,name,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,Manchester United,David de Gea,Goalkeeper,189.0,32.0,104,0.0,0.0,0.009585,0.0,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.0,0.0,0.069018,0.0,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0


#### 5. Uniqueness: 
Check for and remove duplicate records to prevent skewed analysis results.


In [8]:
df.duplicated().sum()

0

In [66]:
# go to delete duplicates columns

#### 6. Completeness: 
Ensure that no critical data is missing. This might mean checking for null values or required fields that are empty.

We will start by checking the dataset for missing or null values. For this, we can use the isna() method which returns a dataframe of boolean values indicating if a field is null or not. To group all missing values by column, we can include the sum() method.

In [27]:
#Display number missing values per column
df.isna().sum()

team                   0
name                   0
position               0
height                 0
age                    0
appearance             0
goals                  0
assists                0
yellow cards           0
second yellow cards    0
red cards              0
goals conceded         0
clean sheets           0
minutes played         0
days_injured           0
games_injured          0
award                  0
current_value          0
highest_value          0
position_encoded       0
winger                 0
dtype: int64

In [68]:
# go to clean them 

#### 7. Check Accuracy:

Verify that the data is correct and precise. This could involve comparing data samples with known sources or using validation rules.

**The process includes:**
1. Validating the appropriateness of data types for the dataset.
2. Identifying outliers  using established validation  rule

In [9]:
# check columns types 
df.dtypes

team                    object
name                    object
position                object
height                 float64
age                    float64
appearance               int64
goals                  float64
assists                float64
yellow cards           float64
second yellow cards    float64
red cards              float64
goals conceded         float64
clean sheets           float64
minutes played           int64
days_injured             int64
games_injured            int64
award                    int64
current_value            int64
highest_value            int64
position_encoded         int64
winger                   int64
dtype: object

In [5]:
df['age'] = df['age'].astype('int64')

In [49]:
df.head(2)

,team,name,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,Manchester United,David de Gea,Goalkeeper,189.0,32,104,0.0,0.0,0.009585,0.0,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,Manchester United,Jack Butland,Goalkeeper,196.0,30,15,0.0,0.0,0.069018,0.0,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0


In [6]:
df['winger'] = df['winger'].replace({1: True, 0: False})

In [51]:
df.head(2)

,team,name,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,Manchester United,David de Gea,Goalkeeper,189.0,32,104,0.0,0.0,0.009585,0.0,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,False
1,Manchester United,Jack Butland,Goalkeeper,196.0,30,15,0.0,0.0,0.069018,0.0,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,False


In [17]:
df.dtypes

team                              object
name                              object
position                          object
height                           float64
age                                int64
appearance                         int64
goals                            float64
assists                          float64
yellow cards                     float64
second yellow cards              float64
red cards                        float64
goals conceded                   float64
clean sheets                     float64
minutes played                     int64
days_injured                       int64
games_injured                      int64
award                              int64
Valuated_price (Euro)              int64
Highest_valuated_price (Euro)      int64
position_encoded                   int64
winger                              bool
dtype: object

In [33]:
# go to clean them 

In [3]:
# check outliers 

### Correcting errors

-------------------

In [7]:
df.rename(columns={
    'current_value': 'Valuated_price (Euro)',
    'highest_value': 'Highest_valuated_price (Euro)'
}, inplace=True)

In [ ]:
# go back to 7th dimension Accuracy 

### Dealing with outliers:

In [18]:
df.dtypes

team                              object
name                              object
position                          object
height                           float64
age                                int64
appearance                         int64
goals                            float64
assists                          float64
yellow cards                     float64
second yellow cards              float64
red cards                        float64
goals conceded                   float64
clean sheets                     float64
minutes played                     int64
days_injured                       int64
games_injured                      int64
award                              int64
Valuated_price (Euro)              int64
Highest_valuated_price (Euro)      int64
position_encoded                   int64
winger                              bool
dtype: object

In [26]:
categorical_features = ['team', 'position', 'position_encoded', 'winger']
numeric_features = ['height', 'age', 'appearance', 'goals', 'assists', 'yellow cards', 'red cards', 'minutes played', 'days_injured', 'award']

In [21]:
filtered_df = df[(df['height'] >= 165) & 
                  (df['height'] <= 200)]

fig = px.histogram(filtered_df, 
                   x='height', 
                   labels={'height': 'height'},
                   nbins=100)

fig.show()

In [9]:
df = df[(df['height'] >= 165) & (df['height'] <= 200)]

In [10]:
df.shape

(10689, 21)

In [24]:
filtered_df = df[(df['age'] >= 17) & 
                  (df['age'] <= 40)]

fig = px.histogram(filtered_df, 
                   x='age', 
                   labels={'age': 'age'},
                   nbins=100)

fig.show()

In [11]:
df = df[(df['age'] >= 17) & (df['age'] <= 40)]

In [12]:
df.shape

(10651, 21)

In [99]:
fig = px.box(df, y = 'Highest_valuated_price (Euro)')
fig.show()

In [34]:
filtered_df = df[(df['Highest_valuated_price (Euro)'] > 450000) & 
                  (df['Highest_valuated_price (Euro)'] <= 20000000)]

fig = px.histogram(filtered_df, 
                   x='Highest_valuated_price (Euro)', 
                   labels={'Highest_valuated_price (Euro)': 'Highest_valuated_price (Euro)'},
                   nbins=100)

fig.show()

In [13]:
df = df[(df['Highest_valuated_price (Euro)'] > 450000) & (df['Highest_valuated_price (Euro)'] <= 20000000)]

In [14]:
df.shape

(7125, 21)

In [37]:
fig = px.box(df, x = 'Valuated_price (Euro)')
fig.show()

In [38]:
filtered_df = df[(df['Valuated_price (Euro)'] >= 450000) & 
                  (df['Valuated_price (Euro)'] <= 200000000)]

fig = px.histogram(filtered_df, 
                   x='Valuated_price (Euro)', 
                   labels={'Valuated_price (Euro)': 'Valuated Price (Euro)'},
                   nbins=100)

fig.show()

In [15]:
df = df[(df['Valuated_price (Euro)'] >= 450000) & (df['Valuated_price (Euro)'] <= 20000000)]

In [16]:
df.shape

(6095, 21)

2.Feature engineering

In [41]:
df.head(2)

,team,name,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,Valuated_price (Euro),Highest_valuated_price (Euro),position_encoded,winger
2,Manchester United,Tom Heaton,Goalkeeper,188.0,37,4,0.0,0.0,0.000000,0.0,...,0.616438,0.924658,292,697,84,4,600000,6000000,1,False
7,Manchester United,Phil Jones,Defender Centre-Back,185.0,31,8,0.0,0.0,0.216346,0.0,...,0.000000,0.000000,416,932,169,7,2000000,20000000,2,False


In [27]:
categorical_features

['team', 'position', 'position_encoded', 'winger']

In [35]:
df.drop(["name"],axis=1,inplace=True)

In [31]:
df = pd.get_dummies(df, columns=categorical_features)

In [32]:
df.shape

(6095, 412)

3. Feature selection

In [36]:
correlation = df.corr()
print(correlation['Valuated_price (Euro)'].sort_values(ascending=False))

Valuated_price (Euro)            1.000000
Highest_valuated_price (Euro)    0.755563
appearance                       0.333725
minutes played                   0.330551
team_Southampton FC              0.113396
                                   ...   
team_Mamelodi Sundowns FC       -0.036952
goals conceded                  -0.044396
position_encoded_1              -0.045106
position_Goalkeeper             -0.045106
age                             -0.185551
Name: Valuated_price (Euro), Length: 411, dtype: float64


In [46]:
threshold = 0.1 # You can change this value based on your requirement

# Filter the correlations
# We use `abs()` for absolute value to consider both strong positive and negative correlations
selected_features = correlation[abs(correlation['Valuated_price (Euro)']) > threshold]['Valuated_price (Euro)'].index
selected_features

Index(['age', 'appearance', 'goals', 'minutes played', 'Valuated_price (Euro)',
       'Highest_valuated_price (Euro)', 'team_Brentford FC',
       'team_Olympique Marseille', 'team_Southampton FC',
       'team_Stade Rennais FC'],
      dtype='object')

In [47]:
selected_features = ['age', 'appearance', 'goals', 'minutes played', 'Valuated_price (Euro)',
       'Highest_valuated_price (Euro)', 'team_Brentford FC',
       'team_Olympique Marseille', 'team_Southampton FC',
       'team_Stade Rennais FC']

In [48]:
df = df[selected_features]
df.head()

,age,appearance,goals,minutes played,Valuated_price (Euro),Highest_valuated_price (Euro),team_Brentford FC,team_Olympique Marseille,team_Southampton FC,team_Stade Rennais FC
2,37,4,0.000000,292,600000,6000000,False,False,False,False
7,31,8,0.000000,416,2000000,20000000,False,False,False,False
8,21,34,0.000000,2758,2000000,2000000,False,False,False,False
11,22,34,0.000000,2686,10000000,12000000,False,False,False,False
20,20,39,0.129125,2788,800000,800000,False,False,False,False


4. Prepare train and test data

In [50]:
# Prepare data
X = df.drop(['Valuated_price (Euro)'], axis=1)
y = df['Valuated_price (Euro)']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=42)

# sacle the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [51]:
X.shape

(6095, 9)

Buliding the Model

In [52]:
model = LinearRegression()

In [54]:
ridge_model = Ridge()

In [55]:
lasso_model = Lasso()

In [56]:
param_grid = {
    'alpha': [0, 0.0003, 0.0002, 0.001, 0.01, 0.5, 1, 5]  # Regularization strength
}
grid_search = GridSearchCV(estimator=ridge_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error', 
                           verbose=1)

grid_search_lasso = GridSearchCV(estimator=lasso_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error', 
                           verbose=1)

Train the Model

In [57]:
model.fit(X_train_scaled, y_train)

LinearRegression()

Test the Model

In [58]:
y_pred = model.predict(X_test_scaled)

Interpretation of the Model

In [59]:
coeff_df = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])
coeff_df

,Coefficient
age,-1.124514e+06
appearance,-9.324172e+05
goals,1.865908e+05
minutes played,1.460947e+06
Highest_valuated_price (Euro),2.631260e+06
team_Brentford FC,2.162021e+05
team_Olympique Marseille,1.226644e+05
team_Southampton FC,1.306636e+05
team_Stade Rennais FC,1.422932e+05


In [60]:
print(model.intercept_) 

2935331.214109926


In [86]:
coef_df = pd.DataFrame(lasso_model.coef_,X.columns,columns=['Coefficient'])
coef_df

AttributeError: 'Lasso' object has no attribute 'coef_'

Evaluating the Model

In [87]:
y_base = [y_train.mean()] * len(y_test)

In [88]:
mse_base = mean_squared_error(y_test, y_base )
print(f'Mean Squared Error of base model: {mse_base}')

Mean Squared Error of base model: 12843588402792.242


In [89]:
mse_train = mean_squared_error(y_train, model.predict(X_train_scaled))
mse_test = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for training: {mse_train}')
print(f'Mean Squared Error for testing: {mse_test}')

Mean Squared Error for training: 3870376727549.9346
Mean Squared Error for testing: 3951656590972.5225


In [90]:
mse_base = mean_absolute_error(y_test, y_base )
print(f'Mean Absolute  Error of base model: {mse_base}')

Mean Absolute  Error of base model: 2444168.819874815


In [91]:
mae_train = mean_absolute_error(y_train, model.predict(X_train_scaled))
mae_test = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error for training: {mae_train}')
print(f'Mean Absolute Error for testing: {mae_test}')

Mean Absolute Error for training: 1355634.1659018681
Mean Absolute Error for testing: 1367879.995432123


In [92]:
print(f'Root Mean Squared  Error of base model: {np.sqrt(mse_base)}')

Root Mean Squared  Error of base model: 1563.3837724227583


In [93]:
print(f'Root Mean Squared Error for training: {np.sqrt(mse_train)}')
print(f'Root Mean Squared Error for testing: {np.sqrt(mse_test)}')

Root Mean Squared Error for training: 1967327.305648436
Root Mean Squared Error for testing: 1987877.4084365773


In [94]:
r2_score_train = r2_score(y_train, model.predict(X_train_scaled))
r2_score_test = r2_score(y_test, y_pred)
print(f'R Square for training: {r2_score_train}')
print(f'R Square for testing: {r2_score_test}')

R Square for training: 0.6919993164956678
R Square for testing: 0.6921211931360791


In [95]:
predictions = model.predict(X_test_scaled)

In [96]:
Real_Values = np.array(y_test)

In [125]:
fig = go.Figure(data=go.Scatter(
    x=Real_Values,
    y=predictions,
    mode='markers'
))

fig.show()

In [134]:
residuals = y_test - predictions
fig = go.Figure(data=[go.Histogram(x=residuals, nbinsx=50)])

fig.show()